# Inteligencia ArtificialProyecto #2: Clasificación
## Inegrantes
    - Luis Berrospi
    - Pedro Dominguez
    - Carlos Esteban Guerrero Robles

### Lectura de datos

In [12]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier, NeighborhoodComponentsAnalysis
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [7]:
df_test = pd.read_csv("..\data\sign_mnist_test.csv")
df_train = pd.read_csv("..\data\sign_mnist_train.csv")

(7172, 785)

### Reducción de dimensionalidad

In [11]:
print(df_train.shape)
print(df_test.shape)
random_state = 0
pca = make_pipeline(StandardScaler(), PCA(n_components=2, random_state=random_state))

(27455, 785)
(7172, 785)


### SVM

#### Estimación del error

#### Tabla de resumen

### KNN

In [13]:
knn = KNeighborsClassifier(n_neighbors=24)
pca.fit(df_train[not df_train.label], df_train[df_train.label])

NameError: name 'n_neighbors' is not defined

#### Estimación del error

#### Tabla de resumen

### Decision tree

#### Estimación del error

#### Tabla de resumen